This script visualizes tactile and visual event based data. 

In [56]:
# add VT_SNN directory
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../../VT_SNN/")

In [57]:
from dataset import ViTacMMDataset
import slayerSNN as snn
import numpy as np
from IPython.display import HTML
import torch
from pathlib import Path

In [60]:
# please indicate data dir
data_dir = '/home/tasbolat/some_python_examples/data_VT_SNN_new/'
train_dataset = torch.load(Path(data_dir) / "ds_vis.pt")

In [67]:
def get_data(num):
    # this function outputs input tactile, vision and downsampled vision data
    vis = torch.FloatTensor(np.load(Path(data_dir) / f'{num:d}_vis.npy'))
    tac = torch.FloatTensor(np.load(Path(data_dir) / f'{num:d}_tact.npy'))
    return tac, vis, train_dataset[num]

In [75]:
tac, vis, ds_vis = get_data(0)
tac.shape, vis.shape, ds_vis.shape

(torch.Size([78, 2, 325]),
 torch.Size([2, 250, 200, 325]),
 torch.Size([2, 63, 50, 325]))

### Event based vision data

In [72]:
# original vision
anim = snn.io.animTD(snn.io.spikeArrayToEvent(vis.numpy()))

HTML(anim.to_jshtml())

In [73]:
# downsampled vision
anim = snn.io.animTD(snn.io.spikeArrayToEvent(ds_vis.numpy()))
HTML(anim.to_jshtml())

### Event based tactile

TODO!

### Generating gifs

In [105]:
import imageio
from PIL import Image

In [144]:
def generate_gif(spike_data, name, resize=True):
    C, H, W, T = spike_data.size()

    red = spike_data.numpy()[0]*255
    red = red.reshape([1,H, W,T])
    green = np.zeros([1,H, W,T])
    blue = spike_data.numpy()[1]*T
    blue = blue.reshape([1,H, W,T])
    rgb = np.concatenate([red, green, blue]).astype(np.uint8)
    gif_ims = []
    for t in range(0,T,1):
        frame = Image.fromarray(rgb[:,:,:,t].transpose(1,2,0))
        frame = frame.rotate(180)
        if resize:
            frame = frame.resize((H*4,W*4), Image.ANTIALIAS)
        gif_ims.append(frame)
        
    kargs = { 'duration': 0.02 }
    imageio.mimsave(name+'.gif', gif_ims, **kargs)

In [160]:
### Generate for each object
tac, vis, ds_vis = get_data(0) # pepsi
tac, vis, ds_vis = get_data(101) # tuna
tac, vis, ds_vis = get_data(201) # soy
tac, vis, ds_vis = get_data(301) # coffee

In [161]:
generate_gif(vis, 'coffee', resize=False)
generate_gif(ds_vis, 'coffee_ds', resize=False)
generate_gif(ds_vis, 'coffee_ds_expanded', resize=True)

### Show generated images

In [178]:
from IPython.display import HTML, display
html_str = "<table>\
<tr><td>Pepsi</td><td>Tuna<td>Soymilk<td>Coffee</td></tr>\
<tr><td><img src='pepsi.gif'></td><td><img src='tuna.gif'><td><img src='soy.gif'><td><img src='coffee.gif'></td></tr>\
<tr><td>Pepsi downsampled</td><td>Tuna downsampled<td>Soymilk downsampled<td>Coffee downsampled</td></tr>\
<tr><td><img src='pepsi_ds.gif'></td><td><img src='tuna_ds.gif'><td><img src='soy_ds.gif'><td><img src='coffee_ds.gif'></td></tr>\
<tr><td>Pepsi downsampled (resized)</td><td>Tuna downsampled (resized)<td>Soymilk downsampled (resized)<td>Coffee downsampled (resized)</td></tr>\
<tr><td><img src='pepsi_ds_expanded.gif'></td><td><img src='tuna_ds_expanded.gif'><td><img src='soy_ds_expanded.gif'><td><img src='coffee_ds_expanded.gif'></td></tr>\
</table>"

display(HTML(html_str))

Pepsi,TunaSoymilkCoffee
,
Pepsi downsampled,Tuna downsampledSoymilk downsampledCoffee downsampled
,
Pepsi downsampled (resized),Tuna downsampled (resized)Soymilk downsampled (resized)Coffee downsampled (resized)
,
